In [1]:
%tensorflow_version 1.x
!pip install tf_slim

TensorFlow 1.x selected.
     |████████████████████████████████| 358kB 4.7MB/s 


In [2]:
import tensorflow as tf
tf.__version__

'1.15.2'

# Convert TF model to OpenVINO 20.01 Intermediate Representation (IR)

This can be used to run inference on OpenVINO.
# In order to run the model on DepthAI modules, we then compile the IR obtained above to a .blob (via a server we set up just for that)



#Install OpenVINO 20.01

In [3]:
%%time
%%capture
## install tools. Open Vino takes some time to download: 10-15 min sometimes.
!sudo apt-get install -y pciutils cpio
!sudo apt autoremove
## downnload installation files
!wget http://registrationcenter-download.intel.com/akdlm/irc_nas/16345/l_openvino_toolkit_p_2020.1.023.tgz
path = "l_openvino_toolkit_p_2020.1.023.tgz"
# path = "/content/software/Intel OpenVINO 2019 R3.1/l_openvino_toolkit_p_2019.3.376.tgz"
## install openvino
!tar xf "{path}"
# !tar xf "{path}" && \
#     cd l_openvino_toolkit_p* && \
#     ./install_openvino_dependencies.sh && \
#     sed -i 's/decline/accept/g' silent.cfg && \
#     ./install.sh
# ## install dependencies
# !/opt/intel/openvino/install_dependencies/install_openvino_dependencies.sh
# ## install prerequisites
# !/opt/intel/openvino/deployment_tools/model_optimizer/install_prerequisites/install_prerequisites.sh

CPU times: user 50.6 ms, sys: 23.9 ms, total: 74.5 ms
Wall time: 18.5 s


In [4]:
%%capture
%cd l_openvino_toolkit_p_2020.1.023/
!./install_openvino_dependencies.sh && \
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg

Open Vino install check

## Convert TF model to Open Vino Intermediate Representation
If using own model, please change to your desired name for output directory --output_dir "choose name"

#According to instructions, the output_dir name is changed to the relevant: medmask_IRv10_10k. I like to use the number of steps trained in the file name.

In [5]:
%cd /opt/intel/openvino_2020.1.023/deployment_tools/model_optimizer/install_prerequisites/
!ls
!./install_prerequisites_tf.sh

/opt/intel/openvino_2020.1.023/deployment_tools/model_optimizer/install_prerequisites
install_prerequisites_caffe.sh	install_prerequisites_onnx.sh
install_prerequisites_kaldi.sh	install_prerequisites.sh
install_prerequisites_mxnet.sh	install_prerequisites_tf.sh
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
H

In [26]:
import tensorflow as tf
tf.__version__

from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

from keras import backend as K
from keras.models import load_model

def export_keras_to_tf(keras_model, output_model, num_output):

    predictions = [None] * num_output
    predrediction_node_names = [None] * num_output

    for i in range(num_output):
        predrediction_node_names[i] = 'output_node' + str(i)
        predictions[i] = tf.identity(keras_model.outputs[i], name=predrediction_node_names[i])

    sess = K.get_session()

    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), predrediction_node_names)
    infer_graph = graph_util.remove_training_nodes(constant_graph) 

    graph_io.write_graph(infer_graph, '.', output_model, as_text=False)


In [44]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

resolution = (500,500)

model = keras.Sequential()
model.add(keras.Input(shape=(resolution[0], resolution[1], 3))) 
model.add(layers.GlobalMaxPool2D())
model.add(layers.Dense(5, activation=tf.nn.softmax))

# model = keras.Sequential()
# model.add(layers.Flatten(input_shape=(resolution[0],resolution[1],3)))
# model.add(layers.Dense(1, activation=tf.nn.softmax))

model.summary()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

output_model = '/content/dumb_model.pb'
predrediction_node_names = export_keras_to_tf(model, output_model, 1)


#tf.saved_model.save(model,'/content/dumb_model') 

print(np.squeeze(model.predict(np.zeros((1,resolution[0],resolution[1],3)))))

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_max_pooling2d_16 (Glo (None, 3)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 20        
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
[0.2 0.2 0.2 0.2 0.2]


In [45]:
#CONVERT TF MODEL to OPEN VINO IRv10. saved in IRv10 directory or choose name
shape_text = "[1,{},{},3]".format(resolution[0],resolution[1])
!source /opt/intel/openvino/bin/setupvars.sh && \
    python /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --input_model "/content/dumb_model.pb" --input_shape {shape_text}  --data_type FP16
    

[setupvars.sh] OpenVINO environment initialized
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/content/dumb_model.pb
	- Path for generated IR: 	/opt/intel/openvino_2020.1.023/deployment_tools/model_optimizer/install_prerequisites/.
	- IR output name: 	dumb_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,500,500,3]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update

## Now we compile the IR model to a .blob for use on DepthAI modules/platform

### We save the blob in the IR directory from above, corresponding to --output_dir parameter above. 
The blob filename will be *frozen_inference_graph.blob*

In [46]:
#CHOOSE the directory where you would like to save the blob.
# I use the same --output_dir as above for the IR conversion
blob_dir = "/content/"

#Copy the path of your .xml and .bin files. For that, you can look at the IR
#conversion output cell, select and copy from:
#[SUCCESS] XML file and bin file paths.
#Or you can choose to compile other .xml .bin files from a different location
#
xmlfile = "/opt/intel/openvino_2020.1.023/deployment_tools/model_optimizer/install_prerequisites/dumb_model.xml"
binfile = "/opt/intel/openvino_2020.1.023/deployment_tools/model_optimizer/install_prerequisites/dumb_model.bin"

import requests

#For openvino 20.01 use this link to compile the blobb
url = "http://69.164.214.171:8080"


#open vino 20.02 link:
# url = "69.164.214.171:8081"

payload = {'compiler_params': '-ip U8 -VPU_MYRIAD_PLATFORM VPU_MYRIAD_2480 -VPU_NUMBER_OF_SHAVES 4 -VPU_NUMBER_OF_CMX_SLICES 4'}
files = [
  ('definition', open(xmlfile,'rb')),
  ('weights', open(binfile,'rb'))
]
# headers = {
#   'Content-Type': 'application/json'
# }
response = requests.request("POST", url, data = payload, files = files)
blobnameraw = response.headers.get('Content-Disposition')
blobname = blobnameraw[blobnameraw.find('='):][1:]
with open(blob_dir + blobname, 'wb') as f:
  f.write(response.content)